<h2><b><i>Radial Basis Function Neural Network</i></b></h2>

Σκοπός της παρούσας άσκησης είναι η κατηγοριοποίηση χειρόγραφων ψηφίων από το [MNIST dataset](https://www.wikiwand.com/en/MNIST_database) με τη χρήση νευρωνικών δικτύων [RBF](https://en.wikipedia.org/wiki/Radial_basis_function) (RBF). Για την κατηγοριοποίηση των χειρόγραφων ψηφίων, χρησιμοποιείται ο αλγόριθμος K-means. Στη κλασική υλοποίηση του K-means όπου έχουμε το κέντρο του cluster και ταξινομομούμε κάθε σημείο με βάση την απόσταση από τα κέντρα. Σε αυτή την περίπτωση μπορεί να υπάρξουν σημεία που να μην μπορούν να αντιστοιχηθούν σε κάποιο cluster. Με τη χρήση του νευρωνικού δικτύου RBF κάθε σημείο ταξινομείται σε ομάδες με συγκεκριμένο κέντρο, γνωρίζοντας όμως τις αποστάσεις μεταξύ των κέντρων αλλά και με διαστήματα εμπιστοσύνης ως προς την πρόβλεψή μας. Με αυτό το τρόπο όλα τα σημεία αντιστοιχίζονται σε ένα cluster. Για να είναι ομαλή η ταξινόμηση των στοιχείων στα clusters μπορεί να χρησιμοποιηθεί μια εκθετική συνάρτηση υψωμένη στην αρνητική τιμή της απόστασης ως γινόμενο με έναν βαθμωτό συντελεστή beta, όπως φαίνεται παρακάτω:

![alt text](https://miro.medium.com/max/145/1*MIay3aIlpT18yewOfnvTiQ.png)

Στα πλαίσια του παραδείγματος για το β χρησιμοποιήθηκε ο παρακάτω τύπος

![alt text](https://miro.medium.com/max/608/1*YEcI_P6orY917fQrzHQEjQ.png)

Όπου k ο αριθμός των clusters και Dmax η μέγιστη ευκλείδια απόσταση μεταξύ των κέντρων. Το dataset που θα χρησιμοποιήσετε είναι διαθέσιμο [εδώ](https://drive.google.com/file/d/1WrVPB3C1vIApWNz5BaSuNmHFopf5htCg/view?usp=sharing) (θα πρέπει να το φορτώσετε στο Colab για να το τρέξετε). 

*Οι ερωτήσεις της άσκησης είναι μαζί με το αντίστοιχο βήμα στον κώδικα*


**Βήμα 1ο**

Τρέχετε τον κώδικα στον οποίο και ορίζονται οι συναρτήσεις που θα μας χρειαστούν. Στα πλαίσια της άσκησης χρησιμοποιούμε μία παραμετροποιημένη έκδοση της συνάρτησης K-Means, όπου μας επιστρέφει τα κέντρα των ομάδων (clusters) καθώς και την τυπική απόκλιση τους.

In [2]:
"""''

Copyright © 2020 Tarlan Ahadli

Permission is hereby granted, free of charge, to any person obtaining a
copy of this software and associated documentation files (the “Software”),
to deal in the Software without restriction, including without limitation
the rights to use, copy, modify, merge, publish, distribute, sublicense,
and/or sell copies of the Software, and to permit persons to whom the Software
is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included
in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY
KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE
WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE
AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT
HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY,
WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR
OTHER DEALINGS IN THE SOFTWARE.

''"""

import numpy as np


def get_distance(x1, x2):
    sum = 0
    for i in range(len(x1)):
        sum += (x1[i] - x2[i]) ** 2
    return np.sqrt(sum)


def kmeans(X, k, max_iters):
    centroids = X[np.random.choice(range(len(X)), k, replace=False)]
    # centroids = [np.random.uniform(size=len(X[0])) for i in range(k)]

    converged = False
    current_iter = 0

    while (not converged) and (current_iter < max_iters):

        cluster_list = [[] for i in range(len(centroids))]

        for x in X:  # Go through each data point
            distances_list = []
            for c in centroids:
                distances_list.append(get_distance(c, x))
            cluster_list[int(np.argmin(distances_list))].append(x)

        cluster_list = list((filter(None, cluster_list)))

        prev_centroids = centroids.copy()

        centroids = []

        for j in range(len(cluster_list)):
            centroids.append(np.mean(cluster_list[j], axis=0))

        pattern = np.abs(np.sum(prev_centroids) - np.sum(centroids))

        print('K-MEANS: ', int(pattern))

        converged = (pattern == 0)

        current_iter += 1

    return np.array(centroids), [np.std(x) for x in cluster_list]


**Βήμα 2ο**

Τρέχουμε την κλάση για το RBFNN

***Ερώτηση 1***

Να αναφέρετε επιγραμματικά με ποιον τρόπο λειτουργεί ένα νευρωνικό δίκτυο RBF.

***Απάντηση 1***
Τα RBF δίκτυα αποτελούνται απο 3 επίπεδα ΙΝPUT - HIDDEN - OUTPUT.
Στο κρυφό επίπεδο (Μοναδικό στα RBF) υπολογίζονται σε κάθε νευρώνα $i$ μια συνάρτηση $h_j(x)$ του input vector $x$.
H συνάρτηση που υπολογίζεται είναι μια **Radial Basis Function** και αντιπροσωπεύει τα χαρακτηριστικά του INPUT vector.
Eπιλέγεται κάποιο κέντρο και τα σημεία τοπθετούνται γύρω απο αυτό. Καθώς απομακρυνόμαστε απο το κέντρο η τιμή της συνάρτησης μειώνεται και μετά απο κάποιο σημείο μηδενίζεται για αυτά τα χ τα οποία βρίσκοντα μετά απο το παραπάνω σημείο. Μια απο τις πιο συχνές συναρτήσεις που χρησιμοποιούνται στα RBF είναι η Γκαουσιανή και συναντάται με το όνομα **RBF kernel**:

$$ h_j(x)  = exp (- \frac {|x - w_j|^2} { 2\sigma_j^2 }) = exp (- \frac { \sum_{1}^{d} ( x_l - w_{jl})^2}{2\sigma_j^2}) $$

Όπου $ W_j = (W_{jl}, ... , W_{jd})^T $ 

καθορίζει το κέντρο και το $j$ καθορίζει την ακτίνα της συνάρτησης για $Μ$ hidden νευρώνες. 
Για input vector: $ x = (x_1,..., x_d)^T$ προκύπτουν τα παρακάτω outputs $ O_i(i=1,...,p)$ : 

$$ o_i(x) = v_{i0} + \sum{j=1}^M  v_{ij}h_j(x)$$

***Ερώτηση 2***

Στα πλαίσια του παραδείγματος, για κάθε αριθμό (κλάση) έχουμε μία μόνο ομάδα (cluster). Τι θα συνέβαινε αν είχαμε παραπάνω από μία ομάδες που θα αντιστοιχούσαν σε μία κλάση; Πώς λύνεται αυτό μέσω του RBF;

***Απάντηση 2***
Για παραπάνω από ένα Cluster για κάθε κλάση δέν θα μπορούσαμε να κατηγοριοποιήσουμε τα δεδομένα μόνο στον ένα cluster για αυτό και η ύπαρξη των παραπάνω βαρών $v_{ij}$. Κατα την εκπαίδευση τα βάρη αυτά μαθαίνονται ώστε να οδηγηθούμε στις κατάλληλες προβλέψεις. 

In [1]:
class RBF:

    def __init__(self, X, y, tX, ty, num_of_classes,
                 k, std_from_clusters=True):
        self.X = X
        self.y = y

        self.tX = tX
        self.ty = ty

        self.number_of_classes = num_of_classes
        self.k = k
        self.std_from_clusters = std_from_clusters

    def convert_to_one_hot(self, x, num_of_classes):
        arr = np.zeros((len(x), num_of_classes))
        for i in range(len(x)):
            c = int(x[i])
            arr[i][c] = 1
        return arr

    def get_rbf(self, x, c, s):
        distance = get_distance(x, c)
        return 1 / np.exp(-distance / s ** 2)

    def get_rbf_as_list(self, X, centroids, std_list):
        RBF_list = []
        for x in X:
            RBF_list.append([self.get_rbf(x, c, s) for (c, s) in zip(centroids, std_list)])
        return np.array(RBF_list)

**Βήμα 3ο**

Τρέχουμε τη συνάρτηση όπου το RBFNN γίνεται fit. Αρχικά τρέχει ο αλγόριθμος k-means για να βρούμε τα κέντρα και την τυπική απόκλιση των clusters. Στη συνέχεια, μπορούμε να χρησιμοποιήσουμε το ίδιο beta για όλα τα clusters


---
***Ερώτηση 3***

Με ποιούς τρόπους μπορούμε να υπολογίσουμε τα βάρη στα hidden layers; Ποιες διαφορές υπάρχουν στους τρόπους αυτούς; Ποιος παρέχει τη καλύτερη δυνατή λύση;

- Tα βάρη υπολογίζονται ανάλογα με την επιλογή του τροπου εκπαίδευσης των δικτύων RBF. Οι κύριοι τρόποι εκπαίδευσης είναι η Εννιαία εκπαίδευση και η εκαπαίδευση σε δύο στάδια. 

>  **1. Εννιαία Εκπαίδευση**:  Χρησιμοποιεί το Gradient Descent για ενημέρωση των βαρών σειριακά για το καθένα ή συνολικά για όλα εννιαία. Μετά την αρχικοποίηση των βαρών, αρχικοποιούνται τα κέντρα και οι radial basis functions. Κακή αρχικοποίηση οδηγεί σε σύγκλιση σε τοπικά ελάχιστα. 

>  **2. Εκπαίδευση σε 2 Στάδια**: 
    
>   - **1ο στάδιο** υπολογίζονται τα πραγματικά κέντρα με βάση τα δεδομένα βασιζόμενοι σε κάποια μέθοδο clustering όπως ο K-means. Αφού υπολογιστούν τα κέντρα η ακτίνα τους υπολογίζεται απο τη διασπορά των δεδομένων γύρω απο κάθε κέντρο. 
$$ \sigma_j^2 = \frac {1}{-N_j} \sum_{1}^{d} |x^n - w_j||^2$$
 > - **2ο Στάδιο**  Τα αρχικά βάρη $w{ij}$ μένουν σταθερά όπως υπολογίστηκαν στο 1ο στάδιο και τα $v_{ij} , v_{i0}$ υπολογίζονται με χρήση Gradient Descent.

In [1]:
  def fit(self):

        self.centroids, self.std_list = kmeans(self.X, self.k, 1000)

        if not self.std_from_clusters:
            dMax = np.max([get_distance(c1, c2) for c1 in self.centroids for c2 in self.centroids])
            self.std_list = np.repeat(dMax / np.sqrt(2 * self.k), self.k)

        RBF_X = self.get_rbf_as_list(self.X, self.centroids, self.std_list)

        self.w = np.linalg.pinv(RBF_X.T @ RBF_X) @ RBF_X.T @ self.convert_to_one_hot(self.y, self.number_of_classes)

        RBF_list_tst = self.get_rbf_as_list(self.tX, self.centroids, self.std_list)

        self.pred_ty = RBF_list_tst @ self.w

        self.pred_ty = np.array([np.argmax(x) for x in self.pred_ty])

        diff = self.pred_ty - self.ty

        print('Accuracy: ', len(np.where(diff == 0)[0]) / len(diff))

**Βήμα 4ο**

Σπάμε το dataset του MNIST σε training και testing και αφήνουμε το RBFNN να βγάλει το τελικό αποτέλεσμα.

---

***Ερώτηση 4***

Επιχειρήστε να αλλάξετε την τιμή του k σε 500 και στη συνέχεια σε 1000. Τι παρατηρείτε ως προς την εκτέλεση;

In [ ]:
##################################
import os 
os.chdir ()

data = np.load('mnist_data.npy').astype(float)

train_y = data[0:2500, 0]
train_x = data[0:2500, 1:]

test_y = data[0:300, 0]
test_x = data[0:300, 1:]

RBF_CLASSIFIER = RBF(train_x, train_y, test_x, test_y, num_of_classes=10,
                     k=10, std_from_clusters=False)

RBF_CLASSIFIER.fit()